## Basic CNN example

---

### Note:
Original source code is from PyTorch Tutorials.  
http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html 

These codes are for python3.  
(Add a bit change, and most of code probably can run with python2)  
Please install Numpy, PyTorch and other libraries before trying to use codes on your computer.  

---

### Contents:
 - Case1 : MNIST classification
 - Case1 : CIFAR10 classification

---

## Case 1 : MNIST classification
  
#### Model
Input : MNIST image, 32x32 pixel  
Output: 10 classes  

Layer | Output
---|---  
Convolution 1 | 6ch @ 28x28  
relu |
maxpooling |
Conv2 | 16ch @ 10x10  
relu |
maxpooling |
FullyConnected 1 | 120  
relu | 
FC2 | 84
relu | 
FC3 | 10  

In [22]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = range(10)

In [25]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(10, 20, 5)
        self.pool2 = nn.MaxPool2d(2, 2)        
        self.fc1 = nn.Linear(320, 160)
        self.fc2 = nn.Linear(160, 80)
        self.fc3 = nn.Linear(80, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        # Convert Output from Convolutional Layer to Input for Fully Connected Layer
        x = x.view(-1, 320)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Net (
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (fc1): Linear (320 -> 160)
  (fc2): Linear (160 -> 80)
  (fc3): Linear (80 -> 10)
)


In [27]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i % 2000 == 1999:
            print("[{0}, {1}] loss: {2}".format(epoch+1, i+1, running_loss/2000))
            running_loss = 0.0

print("Finished Training")

[1, 2000] loss: 1.3947669994251337
[1, 4000] loss: 0.21852607929252554
[1, 6000] loss: 0.12945370480806015
[1, 8000] loss: 0.09928889536356655
[1, 10000] loss: 0.09109163726448787
[1, 12000] loss: 0.08056269900112875
[1, 14000] loss: 0.07246815336299778
[2, 2000] loss: 0.05651079562425798
[2, 4000] loss: 0.053538995732261015
[2, 6000] loss: 0.056236473002662705
[2, 8000] loss: 0.05097658114424098
[2, 10000] loss: 0.05185276292435094
[2, 12000] loss: 0.057244010252028375
[2, 14000] loss: 0.0515387636778471
Finished Training
